# Advanced Model 
Gradient boosting decision tree

In [7]:
from pathlib import Path
import os
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report,roc_curve, auc, RocCurveDisplay, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, StratifiedShuffleSplit
from sklearn.utils import resample

In [11]:
from feature_selection.feature_selection import feature_selection, feature_selection_for_detection
from splitter import splitter

In [9]:
pred_path = "../files/SeizurePrediction/"
dect_path = "../files/SeizureDetection/"

## Prediction

In [4]:
file_list = list(Path(pred_path).glob("*.csv"))
samples = [f.name.split('.')[0] for f in file_list]

In [5]:
file_list[0].parent

WindowsPath('../files/SeizurePrediction')

In [6]:
res = pd.DataFrame(data_x)
res["result"] = data_y
res

NameError: name 'data_x' is not defined

In [16]:
file_list = [f for f in Path(dect_path).glob("*atient*.csv") if "reduced" not in f.name]
for f in file_list:
    # data = pd.read_csv(f.resolve(), header=None)
    sample = f.name.split('.')[0]
    print('='*10,sample,'='*10)
    data_x,data_y = feature_selection_for_detection(f.resolve())
    res = pd.DataFrame(data_x)
    res["result"] = data_y
    res.to_csv(os.path.join(f.parent.resolve(), sample+"_reduced.csv"), index=False)

========== Patient_1 ==========
Number of Observation(raw): 1044
Number of features(raw): 1905
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_2 ==========
Number of Observation(raw): 3141
Number of features(raw): 449
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_3 ==========
Number of Observation(raw): 1041
Number of features(raw): 1541
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_4 ==========
Number of Observation(raw): 210
Number of features(raw): 2017
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_5 ==========
Number of Observation(raw): 2745
Number of features(raw): 1793
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_6 ==========
Number of Observation(raw): 2997
Number of features(raw): 841
Fitting 5 folds for each of 20 candidates, totalling 100 fits
========== Patient_7 ==========
Number of Observation(raw): 3

In [ ]:
file_list = list(Path(pred_path).glob("*.csv"))
for f in file_list:
    # data = pd.read_csv(f.resolve(), header=None)
    sample = f.name.split('.')[0]
    print('='*10,sample,'='*10)
    data_x,data_y = feature_selection(f.resolve())
    res = pd.DataFrame(data_x)
    res["result"] = data_y
    res.to_csv(os.path.join(f.parent.resolve(), sample+"_reduced.csv"), index=False)

In [ ]:
def plot_roc():
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots()
    for i, (train, test) in enumerate(cv.split(X, y)):
        classifier.fit(X[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="Receiver operating characteristic example",
    )
    ax.legend(loc="lower right")
plt.show()

In [49]:
def train_pred(data_x, data_y, plotflag=False, save=False):
    for i in range(5):
        X_train, X_test, y_train, y_test = splitter(data_x, data_y, split_ratio=0.2, scaler=True)
        gradient_boosting = GradientBoostingRegressor(n_estimators=200).fit(X_train, y_train)
        y_pred = gradient_boosting.predict(X_test)
        print(gradient_boosting.score(X_test, y_test))
        print(accuracy_score(y_test, 1*(y_test>0.5)))
        break
    return y_test, y_pred

In [50]:
y_test, y_pred = train_pred(data_x, data_y)

0.048694162192189694
1.0


In [67]:
tn, fp, fn, tp = confusion_matrix(y_test, 1*(y_pred>0.5)).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tp+tn)/(tp+fn+fp+tn)
PPV = tp / (tp+fp)
[specificity,sensitivity,accuracy,PPV]

[0.998062015503876, 0.045454545454545456, 0.9591078066914498, 0.5]

In [68]:
X_train, X_test, y_train, y_test = splitter(data_x, data_y, split_ratio=0.2, scaler=True)

In [72]:
len(data_y)

2688

In [80]:
sss = StratifiedShuffleSplit(train_size=0.8, n_splits=5) 
for train_index, test_index in sss.split(data_x, data_y):
    print(sum(data_y[train_index]), len(train_index))

102.0 2150
102.0 2150
102.0 2150
102.0 2150
102.0 2150
